# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [2]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [3]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [4]:
bootstrap_idx(X_train)

array([  7,  27,  27, 103,  13,  53,  54, 109,  90,  15, 107,  20,  75,
        77,  60,  92,   6,  25,  61,  66,  96,  17,  76,  51,  47, 101,
        55,  16,  66,  14,  86, 107,  39,  92,  80,  21,  43,  56,  58,
        70,  59,  15,  51,  28,  56,  11,  36,  98,  46, 109,  44,  92,
        21,  39,  30,  69,  19,  26,  24,  25,  69,  96,  60,  59,  71,
         4,  57,  77,  98,  76,  18,   6,  42,  91,  98,   8,  41,  13,
        69,  98,  85, 108,  70,  39,  17,  86,  28, 105,  56,   5,   4,
        67,  88,  93,   3,  56,  17,  94,  78,  44, 100,  44, 111,  13,
        80,   9,  67, 105, 100,   2,  35,  20])

In [5]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({41: 3,
         104: 3,
         60: 3,
         56: 2,
         79: 1,
         12: 1,
         110: 1,
         6: 2,
         65: 1,
         59: 1,
         61: 2,
         67: 1,
         54: 1,
         26: 2,
         94: 1,
         23: 1,
         0: 3,
         33: 1,
         63: 3,
         92: 1,
         1: 3,
         14: 1,
         102: 1,
         68: 2,
         75: 1,
         71: 2,
         70: 1,
         90: 1,
         25: 1,
         28: 2,
         95: 1,
         22: 1,
         18: 1,
         98: 1,
         40: 1,
         46: 3,
         47: 3,
         29: 1,
         27: 2,
         96: 3,
         109: 1,
         43: 1,
         8: 1,
         89: 1,
         74: 1,
         73: 2,
         93: 2,
         31: 1,
         66: 3,
         81: 1,
         87: 2,
         78: 3,
         91: 1,
         7: 1,
         82: 1,
         77: 2,
         105: 1,
         20: 1,
         39: 1,
         97: 1,
         5: 1,
         106: 2,
        

In [6]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [7]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [8]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({1: 38, 2: 37, 0: 37})
Classes distribution in the bootstrap: Counter({1: 45, 2: 35, 0: 32})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [9]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test))
    tree_list.append(treei)
    
ypred,_ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))

0.9473684210526315


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

In [13]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    indices = np.random.choice(
        np.arange(X.shape[1]), size=int(np.sqrt(X.shape[1])), replace=False
    )
    X_train_i = X_train_i[:,indices]
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test[:,indices]))
    tree_list.append(treei)
    
ypred,_ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))


0.9473684210526315


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=5)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.868421052631579


In [10]:
from figures import plot_forest_interactive
plot_forest_interactive()

interactive(children=(IntSlider(value=0, description='max_depth', max=8), Output()), _dom_classes=('widget-int…